In [21]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook

In [40]:
Iran_Imps

,Year_Int,Sum of MDollars,HS8_Texted,First Tdesc,Sum of ThTons,Imports,HS6,Condition
0,1400,903.968,10019920,گندم معمولی,2537.988344,903.968,100199,Restricted
1,1395,895.423,85269190,'---سایر دستگاههای کمک ناوبری رادیویی غیر از د...,3.036570,895.423,852691,Takhfif
2,1401,729.991,10019920,گندم معمولی,1589.422554,729.991,100199,Restricted
3,1402,594.598,10019920,گندم معمولی,1570.440177,594.598,100199,Restricted
4,1397,391.407,84014000,اجزاء و قطعات رآکتورهای هسته ای,1.068911,391.407,840140,Takhfif
...,...,...,...,...,...,...,...,...
3496,1397,0.000,85389010,اجزاء و قطعات تعرفه های 8535و8536,0.000094,0.000,853890,Takhfif
3497,1395,0.000,65069900,سایر کلاه ها ، حتی آستر شده یا تزئین شده از سا...,0.000002,0.000,650699,Excluded
3498,1396,0.000,32071010,پیگمانکهای آماده محتوی حداقل 7 درصد فلز طلا ی...,0.000004,0.000,320710,Takhfif
3499,1398,0.000,38241000,چسباننده های آماده برای قالب ها یا مغزی قالب ه...,0.000014,0.000,382410,Takhfif


In [64]:
country = "Russia"
my_path ="C:/TradeGroup/Reports/Eurasia/گزارش/"
Countries = ['Armenia','Belarus','Kazakh','Kyrgyz','Russia']



P174 = pd.read_excel(f'C:/TradeGroup/Reports/Eurasia/P174_Cleaned_Totest.xlsx')
P226 = pd.read_excel(f'C:/TradeGroup/Reports/Eurasia/P226_Cleaned_Totest.xlsx')



Iran_Excluded = list(P174['HS6'].unique())
Iran_Restricted = list(P226['HS6'].unique())
for country in Countries:
    Iran_Imps = pd.read_csv(f'{my_path}Iranfrom{country}.csv')
    Iran_Imps['Sum of MDollars'] = Iran_Imps['Sum of MDollars'].str.replace("$", "")
    Iran_Imps['Imports'] = Iran_Imps['Sum of MDollars'].astype("float")
    Iran_Imps['HS6'] = Iran_Imps['HS8_Texted']//100

    Iran_Imps_Unique_HS6 = Iran_Imps.groupby(["Year_Int","HS6"]).agg(
        MDollars =("Sum of MDollars",'sum'),
        ThTons = ('Sum of ThTons','sum')
        ).reset_index()
    
    Iran_Imps_Unique_HS6['Condition']= np.where(Iran_Imps_Unique_HS6['HS6'].isin(Iran_Excluded),"Excluded",np.where(Iran_Imps_Unique_HS6['HS6'].isin(Iran_Restricted),"Restricted","Takhfif"))

    Iran_Imps_Unique_HS6['Counter'] = 1



    Iran_Imps['Condition']= np.where(Iran_Imps['HS6'].isin(Iran_Excluded),"Excluded",np.where(Iran_Imps['HS6'].isin(Iran_Restricted),"Restricted","Takhfif"))

    Iran_Imps_Condition = Iran_Imps.groupby(['Year_Int','Condition']).agg(
        MDollars = ('Imports','sum'),
        ThTons = ('Sum of ThTons','sum')
        ).reset_index()
    


    Iran_Imps_Condition_Counter = Iran_Imps_Unique_HS6.groupby(['Year_Int','Condition']).agg(
        Counter = ("Counter",'sum')

        ).reset_index()
    


    Iran_Imps_Condition_wide = pd.pivot_table(Iran_Imps_Condition, index='Year_Int', columns='Condition', aggfunc='sum', values=['MDollars', 'ThTons']).reset_index()

    Iran_Imps_Condition_Counter_wide = pd.pivot_table(Iran_Imps_Condition_Counter, index='Year_Int', columns='Condition', aggfunc='sum', values=['Counter']).reset_index()


    Iran_Imps_Condition_wide = Iran_Imps_Condition_wide.merge(Iran_Imps_Condition_Counter_wide,on="Year_Int",how="outer")


    Iran_Imps_Condition_wide.columns = [f"{col[1]}_{col[0]}" for col in Iran_Imps_Condition_wide.columns]


    Iran_Imps_Condition_wide =    Iran_Imps_Condition_wide.fillna(0)

    if "Restricted_MDollars" in Iran_Imps_Condition_wide.columns:
        pass
    else:
        Iran_Imps_Condition_wide['Restricted_MDollars'] = 0
        Iran_Imps_Condition_wide['Restricted_ThTons'] = 0




    Iran_Imps_Condition_wide['Excluded_MDollars_Share'] = Iran_Imps_Condition_wide['Excluded_MDollars']/(Iran_Imps_Condition_wide['Excluded_MDollars']+Iran_Imps_Condition_wide['Restricted_MDollars']+Iran_Imps_Condition_wide['Takhfif_MDollars'])
    Iran_Imps_Condition_wide['Restricted_MDollars_Share'] = Iran_Imps_Condition_wide['Restricted_MDollars']/(Iran_Imps_Condition_wide['Excluded_MDollars']+Iran_Imps_Condition_wide['Restricted_MDollars']+Iran_Imps_Condition_wide['Takhfif_MDollars'])
    Iran_Imps_Condition_wide['Takhfif_MDollars_Share'] = Iran_Imps_Condition_wide['Takhfif_MDollars']/(Iran_Imps_Condition_wide['Excluded_MDollars']+Iran_Imps_Condition_wide['Restricted_MDollars']+Iran_Imps_Condition_wide['Takhfif_MDollars'])



    Iran_Imps_Condition_wide.to_excel(f'C:/TradeGroup/Reports/Eurasia/Iran_Imports{country}.xlsx',index=False)



C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_596\2311098705.py:52: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  Iran_Imps_Condition_wide = Iran_Imps_Condition_wide.merge(Iran_Imps_Condition_Counter_wide,on="Year_Int",how="outer")
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_596\2311098705.py:52: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  Iran_Imps_Condition_wide = Iran_Imps_Condition_wide.merge(Iran_Imps_Condition_Counter_wide,on="Year_Int",how="outer")
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_596\2311098705.py:52: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  Iran_Imps_Condition_wide = Iran_Imps_Condition_wide.merge(Iran_Imps_Condition_Counter_wide,on="Year_Int",how="outer")
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_596\2311098705.py:52: PerformanceWarning: droppi

In [104]:
country = "Russia"
my_path ="C:/TradeGroup/Reports/Eurasia/گزارش/"
Countries = ['Armenia','Belarus','Kazakh','Kyrgyz','Russia']



P118 = pd.read_excel(f'C:/TradeGroup/Reports/Eurasia/P118_Cleaned_Totest.xlsx')
P166 = pd.read_excel(f'C:/TradeGroup/Reports/Eurasia/P166_Cleaned_Totest.xlsx')



They_Excluded = list(P118['HS6'].unique())
They_Restricted = list(P166['HS6'].unique())
for country in Countries:
    Iran_Exps = pd.read_csv(f'{my_path}Iranto{country}.csv')
    Iran_Exps['Sum of MDollars'] = Iran_Exps['Sum of MDollars'].str.replace("$", "")
    Iran_Exps['Imports'] = Iran_Exps['Sum of MDollars'].astype("float")
    Iran_Exps['HS6'] = Iran_Exps['HS8_Texted']//100



    Iran_Exps_Unique_HS6 = Iran_Exps.groupby(["Year_Int","HS6"]).agg(
        MDollars =("Sum of MDollars",'sum'),
        ThTons = ('Sum of ThTons','sum')
        ).reset_index()
    
    Iran_Exps_Unique_HS6['Condition']= np.where(Iran_Exps_Unique_HS6['HS6'].isin(They_Excluded),"Excluded",np.where(Iran_Exps_Unique_HS6['HS6'].isin(They_Restricted),"Restricted","Takhfif"))

    Iran_Exps_Unique_HS6['Counter'] = 1









    Iran_Exps['Condition']= np.where(Iran_Exps['HS6'].isin(They_Excluded),"Excluded",np.where(Iran_Exps['HS6'].isin(They_Restricted),"Restricted","Takhfif"))
    Iran_Exps_Condition = Iran_Exps.groupby(['Year_Int','Condition']).agg(
        MDollars = ('Imports','sum'),
        ThTons = ('Sum of ThTons','sum')
        ).reset_index()
    



    Iran_Exps_Condition_Counter = Iran_Exps_Unique_HS6.groupby(['Year_Int','Condition']).agg(
        Counter = ("Counter",'sum')

        ).reset_index()
    


    Iran_Exps_Condition_Counter_wide = pd.pivot_table(Iran_Exps_Condition_Counter, index='Year_Int', columns='Condition', aggfunc='sum', values=['Counter']).reset_index()

    Iran_Exps_Condition_Counter_wide.columns = Iran_Exps_Condition_Counter_wide.columns.droplevel(0)

    Iran_Exps_Condition_Counter_wide.rename(columns={Iran_Exps_Condition_Counter_wide.columns[0]: '_Year_Int'}, inplace=True)





    


    Iran_Exps_Condition_wide = pd.pivot_table(Iran_Exps_Condition, index='Year_Int', columns='Condition', aggfunc='sum', values=['MDollars', 'ThTons']).reset_index()


    Iran_Exps_Condition_wide.columns = [f"{col[1]}_{col[0]}" for col in Iran_Exps_Condition_wide.columns]

    Iran_Exps_Condition_wide = Iran_Exps_Condition_wide.merge(Iran_Exps_Condition_Counter_wide,on="_Year_Int",how="outer")


    Iran_Exps_Condition_wide =    Iran_Exps_Condition_wide.fillna(0)

    if "Restricted_MDollars" in Iran_Exps_Condition_wide.columns:
        pass
    else:
        Iran_Exps_Condition_wide['Restricted_MDollars'] = 0
        Iran_Exps_Condition_wide['Restricted_ThTons'] = 0




    Iran_Exps_Condition_wide['Excluded_MDollars_Share'] = Iran_Exps_Condition_wide['Excluded_MDollars']/(Iran_Exps_Condition_wide['Excluded_MDollars']+Iran_Exps_Condition_wide['Restricted_MDollars']+Iran_Exps_Condition_wide['Takhfif_MDollars'])
    Iran_Exps_Condition_wide['Restricted_MDollars_Share'] = Iran_Exps_Condition_wide['Restricted_MDollars']/(Iran_Exps_Condition_wide['Excluded_MDollars']+Iran_Exps_Condition_wide['Restricted_MDollars']+Iran_Exps_Condition_wide['Takhfif_MDollars'])
    Iran_Exps_Condition_wide['Takhfif_MDollars_Share'] = Iran_Exps_Condition_wide['Takhfif_MDollars']/(Iran_Exps_Condition_wide['Excluded_MDollars']+Iran_Exps_Condition_wide['Restricted_MDollars']+Iran_Exps_Condition_wide['Takhfif_MDollars'])














    Iran_Exps_Condition_wide.to_excel(f'C:/TradeGroup/Reports/Eurasia/Iran_Exports{country}.xlsx',index=False)

    





Iran_Exps_Condition_wide

,_Year_Int,Excluded_MDollars,Restricted_MDollars,Takhfif_MDollars,Excluded_ThTons,Restricted_ThTons,Takhfif_ThTons,Excluded,Restricted,Takhfif,Excluded_MDollars_Share,Restricted_MDollars_Share,Takhfif_MDollars_Share
0,1394,17.418,41.052,115.046,15.395725,91.512127,327.285214,22,6,187,0.100383,0.236589,0.663028
1,1395,20.771,42.679,155.498,19.780777,111.311910,230.953102,28,6,258,0.094867,0.194928,0.710205
2,1396,31.991,53.237,199.351,27.842502,113.580672,384.540375,36,6,270,0.112415,0.187073,0.700512
3,1397,31.642,62.630,189.581,34.772255,126.619405,341.707396,36,6,287,0.111473,0.220642,0.667884
4,1398,98.152,87.835,272.891,87.978945,153.161038,673.119865,39,5,315,0.213896,0.191413,0.594692
5,1399,57.321,131.388,312.408,100.518115,263.972866,678.637722,41,6,350,0.114386,0.262190,0.623423
6,1400,159.427,50.991,368.164,192.397493,177.407690,770.220154,47,6,369,0.275548,0.088131,0.636321
7,1401,162.189,24.810,564.405,244.287534,87.607346,1104.168872,66,6,498,0.215848,0.033018,0.751134
8,1402,285.738,60.533,618.196,368.676142,115.624599,1730.767852,61,6,556,0.296265,0.062763,0.640972
9,1403,157.665,25.121,311.525,218.779978,37.002316,1033.151190,61,6,432,0.318959,0.050820,0.630221


In [102]:
Iran_Exps_Condition_wide

,_Year_Int,Excluded_MDollars,Restricted_MDollars,Takhfif_MDollars,Excluded_ThTons,Restricted_ThTons,Takhfif_ThTons,Excluded_x,Restricted_x,Takhfif_x,Excluded_y,Restricted_y,Takhfif_y,Excluded,Restricted,Takhfif
0,1394,17.418,41.052,115.046,15.395725,91.512127,327.285214,45,5,465,45,5,465,45,5,465
1,1395,20.771,42.679,155.498,19.780777,111.311910,230.953102,43,6,457,43,6,457,43,6,457
2,1396,31.991,53.237,199.351,27.842502,113.580672,384.540375,56,5,527,56,5,527,56,5,527
3,1397,31.642,62.630,189.581,34.772255,126.619405,341.707396,52,5,645,52,5,645,52,5,645
4,1398,98.152,87.835,272.891,87.978945,153.161038,673.119865,55,6,661,55,6,661,55,6,661
5,1399,57.321,131.388,312.408,100.518115,263.972866,678.637722,56,6,750,56,6,750,56,6,750
6,1400,159.427,50.991,368.164,192.397493,177.407690,770.220154,69,6,845,69,6,845,69,6,845
7,1401,162.189,24.810,564.405,244.287534,87.607346,1104.168872,85,6,844,85,6,844,85,6,844
8,1402,285.738,60.533,618.196,368.676142,115.624599,1730.767852,81,5,860,81,5,860,81,5,860
9,1403,157.665,25.121,311.525,218.779978,37.002316,1033.151190,65,4,689,65,4,689,65,4,689


In [93]:
Iran_Exps_Condition_wide

,_Year_Int,Excluded_MDollars,Restricted_MDollars,Takhfif_MDollars,Excluded_ThTons,Restricted_ThTons,Takhfif_ThTons
0,1394,17.418,41.052,115.046,15.395725,91.512127,327.285214
1,1395,20.771,42.679,155.498,19.780777,111.311910,230.953102
2,1396,31.991,53.237,199.351,27.842502,113.580672,384.540375
3,1397,31.642,62.630,189.581,34.772255,126.619405,341.707396
4,1398,98.152,87.835,272.891,87.978945,153.161038,673.119865
5,1399,57.321,131.388,312.408,100.518115,263.972866,678.637722
6,1400,159.427,50.991,368.164,192.397493,177.407690,770.220154
7,1401,162.189,24.810,564.405,244.287534,87.607346,1104.168872
8,1402,285.738,60.533,618.196,368.676142,115.624599,1730.767852
9,1403,157.665,25.121,311.525,218.779978,37.002316,1033.151190
